Runs the fg transform tool 

For Methylation Reaction (with no optimization)

In [1]:
from fgtransform import statereaction

#Dataset
dataset_filepath = 'data/datasets/QM9/qm9.db'


#H-TYPE TRANSFORMATION
targetlabelH_filepath = 'data/autolabel/Hqm9alllabelpert4/labelpert4ver2.csv'

#pert3
#remove all H's of CH3-CH2-C and evaluate O embs ---> CH2-CH2-C...O (pert 3)
#labelH1id = 17

#pert 1 - 3, pret2 - 9, pert 3 - 21 
#OH -> O for primary alcohol O embedding
labelH1id = 69

n_molecules = [0,10000]

#OUTPUT
output_filepath1 = 'data/fgtransform/model1/remove/H/1hydroxylOpert4/init.xyz'
output_filepath2 = 'data/fgtransform/model1/remove/H/1hydroxylOpert4/trans.xyz'

available_properties = ['energy']

statereaction = statereaction.transform(dataset_filepath,targetlabelH_filepath,labelH1id,output_filepath1,output_filepath2,n_molecules,available_properties)
number_trans = statereaction.removeH()

/home/amerelsamman/anaconda3/envs/dlchem/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/amerelsamman/anaconda3/envs/dlchem/lib/python3.7/site-packages/schnetpack/data/atoms.py:327: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:199.)
  properties[pname] = torch.FloatTensor(prop)
/home/amerelsamman/anaconda3/envs/dlchem/lib/python3.7/site-packages/ase/atoms.py:968: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  np.Visibl

Properties: [{'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtype=float32)}, {'energy': array([0.], dtyp

In [ ]:
print(number_trans)

FGTransform Analysis

#Run extract embeddings on init and trans

In [ ]:
#Run extract embeddings on init and trans
from extractembeddings import allelementextract

init_dataset_filepath = 'data/fgtransform/model1/remove/H/1hydroxylOpert4/init.db'
init_save_filepath = 'data/fgtransform/model1/remove/H/1hydroxylOpert4/initO.csv'
trans_dataset_filepath = 'data/fgtransform/model1/remove/H/1hydroxylOpert4/trans.db'
trans_save_filepath = 'data/fgtransform/model1/remove/H/1hydroxylOpert4/transO.csv'

model_filepath = 'data/trainedmodels/model1/best_model'
start = 0
end = 68
n_features = 128
n_layers = [5,6]
all_elements = False
element = [8]
qm9=False
available_properties = ['energy']
#label_file =  '../../data/labeldataset/%s/label%s%s.csv' %(element,element,number_inputs)

#run extract on init and trans
#default: True
label = False
restrict_label = False
allowed_labels = [8,7]
scratch_file = 'temp2.xyz'


#allelementextract.extract(qm9,init_dataset_filepath,model_filepath,init_save_filepath,start,end,n_features,n_layers,all_elements,element,available_properties=available_properties)
allelementextract.extract(qm9,init_dataset_filepath,model_filepath,init_save_filepath,start,end,n_features,n_layers,all_elements,element,available_properties=available_properties,label=label)


transform with average diff vector and use pca on both

In [ ]:
from tools.utils import utils_fgtransform, utils_dimred, utils_numericalfiletools

n_molecules = 300
n_features = 128
fg_trans_fildir = 'data/fgtransform/model1/remove/H/1hydroxylOpert3/'

#Calculate average diff vector
data_filepath1 = fg_trans_fildir+ 'initO.csv'
data_filepath2 = fg_trans_fildir+ 'transO.csv'
diff_mean = utils_fgtransform.vecdiffmean(data_filepath1,data_filepath2,n_features,n_molecules)


#Perform transformation
vec_filepath = fg_trans_fildir+ 'initOdiff.csv'
save_filepath = fg_trans_fildir+ 'initOtransform.csv'
utils_fgtransform.add_vectomat(vec_filepath,data_filepath1,n_features,save_filepath)

from tools.utils import utils_fgtransform

true_trans_filepath = fg_trans_fildir+ 'transO.csv'
art_trans_filepath = fg_trans_fildir+ 'initOtransform.csv'
#DO NOT SORT! 
utils_fgtransform.nearest_trans(true_trans_filepath,art_trans_filepath,n_molecules,n_features)




In [ ]:
from tools.utils import utils_fgtransform, utils_dimred, utils_numericalfiletools
n_molecules = 68
n_features = 128
fg_trans_fildir = 'data/fgtransform/model1/oxidation/1alcsoxpert2/'


#must stack initOtransform with initO call it bothO
init_filepath = fg_trans_fildir+ 'initO.csv'
inittrans_filepath = fg_trans_fildir+ 'initOtransform.csv'
save_filepath = fg_trans_fildir+ 'bothOtransform.csv'
n_data1 = n_molecules
n_data2 = n_molecules
utils_numericalfiletools.vstacktwofiles(init_filepath,inittrans_filepath,save_filepath,n_data1,n_data2,single_dim=False,skip_header1=1,skip_header2=0)

n_molecules = n_molecules*2
#Dimensionality reduction on new file with init and inittransformed
data_filepath = fg_trans_fildir+ 'bothOtransform.csv'
save_filepath = fg_trans_fildir+ 'initOtransform.csv'
n_components = 128
skip_header =  0
scale_data = False
utils_dimred.pca(data_filepath,n_components,n_features,scale_data,save_filepath, n_molecules,skip_header)

data_filepath = fg_trans_fildir+ 'initOtransformpca.csv'
save_filepath = fg_trans_fildir+ 'initOtransformpcavecs.csv'
#Make vector plotting file from the pca (x1,y1,x2,y2) 
#(half the data is init, half is trans after compilation)
utils_fgtransform.scatters_to_vectors(data_filepath,save_filepath)